In [79]:
!python data_preprocess.py sample-data/eng eng

Traceback (most recent call last):
  File "data_preprocess.py", line 198, in <module>
    triplets = augment(lowi, lowo, lowt, vocab)
  File "data_preprocess.py", line 97, in augment
    i, o = item[0], item[1]
IndexError: list index out of range


In [54]:
inp = 'X'
oup = ['a','b','c','d']

In [67]:
for ind_o, oup_s in enumerate(oup):
    print(ind_o, oup[0:ind_o],oup[ind_o])
    pref = ' '.join(oup[0:ind_o])
    pref = 0 if len(pref)==0 else pref
    print(pref)

0 [] a
0
1 ['a'] b
a
2 ['a', 'b'] c
a b
3 ['a', 'b', 'c'] d
a b c


In [ ]:
def alignment(inputs, outputs, poss, characters):
	temp = [(''.join(inputs[i]), ''.join(outputs[i])) for i in range(len(outputs))]
	aligned = align.Aligner(temp).alignedpairs
	vocab = list(characters)
	try:
		vocab.remove(u" ")
	except:
		pass

	triplets = []
	for k, item in enumerate(aligned):
		inp, oup, pos = ''.join(inputs[k]), ''.join(outputs[k]), '-'.join(poss[k])
		tx_set = [inp, oup, pos]
		i, o = item[0], item[1]
		inp_len, oup_len, al_len = len(inp), len(oup), len(i)
		lss, mb, stc = seqMatch(inp, oup)
		eps = '\u0395'
		prefix_flag = False
		tag = ''.join(list(zip(*stc))[0])
		print(tag)
		if len(inp.split(" ")) < len(oup.split(" ")):
			oups = oup.split(" ")
			for ind_o, oup_s in enumerate(oups):
				pref = ' '.join(oups[0:ind_o])
				pref =  pref if len(pref)>0 else eps
				triplet = make_triplet(tx_set, pref, "{}->{}".format(inp, oup_s), eps)
				triplets.append(triplet)
				triplet = make_triplet(tx_set, pref, "{}->{}".format(inp, oup_s[:-1]), oup[-1:])
				triplets.append(triplet)
				triplet = make_triplet(tx_set, pref, "{}->{}".format(inp, oup_s[:-2]), oup[-2:])


		print("_________")

	return triplets